In [6]:
try:
    !kaggle datasets list
    print("Snippets of Kaggle Datasets Displayed above")
except:
    !pip install -q kaggle
    print("Installation of the Kaggle Done")
    !kaggle datasets list
    print("Snippets of Kaggle Datasets Displayed above")

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           9557        281  1.0              
mattop/alcohol-consumption-per-capita-2016                      Alcohol Consumption Per Capita 2016                   4KB  2022-12-09 00:03:11            989         36  1.0              
michals22/coffee-dataset                                        Coffee dataset                                       24KB  2022-12-15 20:02:12           1114         41  1.0              
thedevastator/jobs-dataset-from-glassdoor                   

In [11]:
# Download The Datasets
!kaggle datasets download -d toobajamal/kdd99-dataset

kdd99-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [15]:
import zipfile
ZIP_FILE_NAME="kdd99-dataset.zip"
zip_ref=zipfile.ZipFile(ZIP_FILE_NAME,'r')
zip_ref.extractall()
zip_ref.close()

In [52]:
# Import libraries and other frameworks

import xgboost as xgb
import numpy as np
from collections import OrderedDict
import os
import pandas as pd
from copy import copy
from time import time
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,classification_report,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pickle

np.random.seed(123)

# Display all columns of Pandas' dataframes by default
pd.set_option('display.max_columns', None)

data_path = 'kddcup99_csv.csv'

In [53]:
df =  pd.read_csv(data_path)

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  lnum_compromised             494020 non-null  int64  
 13 

In [55]:
pd.DataFrame(df['label'].value_counts())

,label
smurf,280790
neptune,107201
normal,97277
back,2203
satan,1589
ipsweep,1247
portsweep,1040
warezclient,1020
teardrop,979
pod,264


In [56]:
# Apply labelencoder 
le = LabelEncoder()
le.fit(df.label)
print(le.classes_)

['back' 'buffer_overflow' 'ftp_write' 'guess_passwd' 'imap' 'ipsweep'
 'land' 'loadmodule' 'multihop' 'neptune' 'nmap' 'normal' 'perl' 'phf'
 'pod' 'portsweep' 'rootkit' 'satan' 'smurf' 'spy' 'teardrop'
 'warezclient' 'warezmaster']


In [57]:
# capture the categorical variables and one-hot encode them
cat_vars = ['protocol_type', 'service', 'flag', 'land', 'logged_in','is_host_login', 'is_guest_login']
cat_data = pd.get_dummies(df[cat_vars])
cat_data.head()

,land,logged_in,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_hostnames,service_http,service_http_443,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,service_name,service_netbios_dgm,service_netbios_ns,service_netbios_ssn,service_netstat,service_nnsp,service_nntp,service_ntp_u,service_other,service_pm_dump,service_pop_2,service_pop_3,service_printer,service_private,service_red_i,service_remote_job,service_rje,service_shell,service_smtp,service_sql_net,service_ssh,service_sunrpc,service_supdup,service_systat,service_telnet,service_tftp_u,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [58]:
numeric_vars = list(set(df.columns.values.tolist()) - set(cat_vars))
numeric_vars.remove('label')
numeric_data = df[numeric_vars].copy()

# check that the numeric data has been captured accurately
numeric_data.head()

,same_srv_rate,srv_rerror_rate,urgent,diff_srv_rate,lnum_file_creations,lsu_attempted,lnum_outbound_cmds,hot,dst_host_diff_srv_rate,count,srv_count,lnum_access_files,lroot_shell,wrong_fragment,lnum_shells,src_bytes,duration,lnum_compromised,dst_host_srv_rerror_rate,serror_rate,dst_host_same_srv_rate,dst_host_count,dst_host_srv_count,dst_host_serror_rate,num_failed_logins,dst_host_same_src_port_rate,srv_serror_rate,dst_host_srv_serror_rate,dst_bytes,dst_host_srv_diff_host_rate,lnum_root,rerror_rate,dst_host_rerror_rate,srv_diff_host_rate
0,1.0,0.0,0,0.0,0,0,0,0,0.0,8,8,0,0,0,0,181,0,0,0.0,0.0,1.0,9,9,0.0,0,0.11,0.0,0.0,5450,0.0,0,0.0,0.0,0.0
1,1.0,0.0,0,0.0,0,0,0,0,0.0,8,8,0,0,0,0,239,0,0,0.0,0.0,1.0,19,19,0.0,0,0.05,0.0,0.0,486,0.0,0,0.0,0.0,0.0
2,1.0,0.0,0,0.0,0,0,0,0,0.0,8,8,0,0,0,0,235,0,0,0.0,0.0,1.0,29,29,0.0,0,0.03,0.0,0.0,1337,0.0,0,0.0,0.0,0.0
3,1.0,0.0,0,0.0,0,0,0,0,0.0,6,6,0,0,0,0,219,0,0,0.0,0.0,1.0,39,39,0.0,0,0.03,0.0,0.0,1337,0.0,0,0.0,0.0,0.0
4,1.0,0.0,0,0.0,0,0,0,0,0.0,6,6,0,0,0,0,217,0,0,0.0,0.0,1.0,49,49,0.0,0,0.02,0.0,0.0,2032,0.0,0,0.0,0.0,0.0


In [59]:
# concat numeric and the encoded categorical variables
numeric_cat_data = pd.concat([numeric_data, cat_data], axis=1)

# here we do a quick sanity check that the data has been concatenated correctly by checking the dimension of the vectors
print(cat_data.shape)
print(numeric_data.shape)
print(numeric_cat_data.shape)

(494020, 84)
(494020, 34)
(494020, 118)


In [60]:
# capture the labels
labels = df['label'].copy()


integer_labels = le.transform(labels)
x_train, x_test, y_train, y_test = train_test_split(numeric_cat_data,integer_labels,test_size=.25,random_state=42)

In [61]:
# check that the dimensions of our train and test sets are okay
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(370515, 118)
(370515,)
(123505, 118)
(123505,)


In [62]:
# save the datasets for later use
preprocessed_data = {
    'x_train':x_train,
    'y_train':y_train,
    'x_test':x_test,
    'y_test':y_test,
    'le':le
}

# pickle the preprocessed_data
path = 'preprocessed_data_full.pkl'
out = open(path, 'wb')
pickle.dump(preprocessed_data, out)
out.close()

In [63]:
def convert_label_to_binary(label_encoder, labels):
    normal_idx = np.where(label_encoder.classes_ == 'normal.')[0]
    my_labels = labels.copy()
    my_labels[my_labels != normal_idx] = 1 
    my_labels[my_labels == normal_idx] = 0
    return my_labels

binary_y_train = convert_label_to_binary(le, y_train)
binary_y_test = convert_label_to_binary(le, y_test)

# check how many anomalies are in our labels
print('Number of anomalies in y_train: ', binary_y_train.sum())
print('Number of anomalies in y_test:  ', binary_y_test.sum())

Number of anomalies in y_train:  370515
Number of anomalies in y_test:   123505


C:\Users\luhar\AppData\Local\Temp\ipykernel_1572\864451187.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  my_labels[my_labels != normal_idx] = 1
C:\Users\luhar\AppData\Local\Temp\ipykernel_1572\864451187.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  my_labels[my_labels == normal_idx] = 0


In [64]:
params = {
    'num_rounds':        10,
    'max_depth':         8,
    'max_leaves':        2**8,
    'alpha':             0.9, # L1 regularization term on weights.
    'eta':               0.1,
    'gamma':             0.1,
    'learning_rate':     0.1,
    'subsample':         1,
    'reg_lambda':        1,
    'scale_pos_weight':  2,
    'tree_method':       'gpu_hist',
    'n_gpus':            1,
    'objective':         'binary:logistic',
    'verbose':           True
}

In [73]:
x_train.head()

,same_srv_rate,srv_rerror_rate,urgent,diff_srv_rate,lnum_file_creations,lsu_attempted,lnum_outbound_cmds,hot,dst_host_diff_srv_rate,count,srv_count,lnum_access_files,lroot_shell,wrong_fragment,lnum_shells,src_bytes,duration,lnum_compromised,dst_host_srv_rerror_rate,serror_rate,dst_host_same_srv_rate,dst_host_count,dst_host_srv_count,dst_host_serror_rate,num_failed_logins,dst_host_same_src_port_rate,srv_serror_rate,dst_host_srv_serror_rate,dst_bytes,dst_host_srv_diff_host_rate,lnum_root,rerror_rate,dst_host_rerror_rate,srv_diff_host_rate,land,logged_in,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_hostnames,service_http,service_http_443,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,service_name,service_netbios_dgm,service_netbios_ns,service_netbios_ssn,service_netstat,service_nnsp,service_nntp,service_ntp_u,service_other,service_pm_dump,service_pop_2,service_pop_3,service_printer,service_private,service_red_i,service_remote_job,service_rje,service_shell,service_smtp,service_sql_net,service_ssh,service_sunrpc,service_supdup,service_systat,service_telnet,service_tftp_u,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
36198,1.0,0.0,0,0.0,0,0,0,0,0.05,1,1,0,0,0,0,1189,0,0,0.0,0.0,0.47,133,146,0.00,0,0.01,0.0,0.00,361,0.01,0,0.0,0.0,0.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
307313,1.0,0.0,0,0.0,0,0,0,0,0.00,511,511,0,0,0,0,1032,0,0,0.0,0.0,1.00,255,255,0.00,0,1.00,0.0,0.00,0,0.00,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
486986,1.0,0.0,0,0.0,0,0,0,24,0.06,1,1,0,0,0,0,1205,64,0,0.0,0.0,0.33,66,22,0.02,0,0.02,0.0,0.05,3380,0.00,0,0.0,0.0,0.0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
206620,1.0,0.0,0,0.0,0,0,0,0,0.00,511,511,0,0,0,0,1032,0,0,0.0,0.0,1.00,255,255,0.00,0,1.00,0.0,0.00,0,0.00,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
300928,1.0,0.0,0,0.0,0,0,0,0,0.00,511,511,0,0,0,0,1032,0,0,0.0,0.0,1.00,255,255,0.00,0,1.00,0.0,0.00,0,0.00,0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [74]:
y_train[0:100]

array([11, 18, 11, 18, 18,  9, 18,  9, 11, 18, 11, 18, 18,  9, 18, 11, 11,
       18, 18, 11, 18, 18,  0, 18, 18,  9, 18, 18, 11, 11, 11, 18,  9, 11,
       18, 11, 18,  9, 20, 18, 11, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18,  9, 18, 11, 18,  9,  9, 18, 18, 18, 18, 11, 18,  9, 18,
       18, 18,  9, 18, 18, 18, 18,  9, 18, 18, 18,  9, 11, 18, 18,  9,  9,
       11, 11,  9, 11, 18,  9, 18, 18, 18, 18, 18, 18, 18, 11, 18])

In [78]:
%%time
dtrain = xgb.DMatrix(x_train, label=binary_y_train)
dtest = xgb.DMatrix(x_test, label=binary_y_test)
evals = [(dtest, 'test',), (dtrain, 'train')]
num_rounds = params['num_rounds']

Wall time: 2.18 s


In [79]:
num_rounds = params['num_rounds']

In [80]:
model = xgb.train(params, dtrain, num_rounds, evals=evals)

[13:16:54] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_gpus", "num_rounds", "verbose" } are not used.

[0]	test-logloss:0.59814	train-logloss:0.59814
[1]	test-logloss:0.52033	train-logloss:0.52033
[2]	test-logloss:0.45545	train-logloss:0.45545
[3]	test-logloss:0.40060	train-logloss:0.40060
[4]	test-logloss:0.35375	train-logloss:0.35375
[5]	test-logloss:0.31339	train-logloss:0.31339
[6]	test-logloss:0.27839	train-logloss:0.27839
[7]	test-logloss:0.24786	train-logloss:0.24786
[8]	test-logloss:0.22110	train-logloss:0.22110
[9]	test-logloss:0.19756	train-logloss:0.19756


In [67]:
threshold = .5
true_labels = binary_y_test.astype(int)
true_labels.sum()

123505

In [68]:
# make predictions on the test set using our trained model
preds = model.predict(dtest)
print(preds)

[0.8207288 0.8207288 0.8207288 ... 0.8207288 0.8207288 0.8207288]


In [70]:
pred_labels = (preds > threshold).astype(int)
print(pred_labels)

[1 1 1 ... 1 1 1]


In [71]:
pred_labels.sum()

123505

In [ ]:
print ('Accuracy:', accuracy_score(true_labels, pred_labels))

In [ ]:
print ('Accuracy:', accuracy_score(true_labels, pred_labels))

In [ ]:
results = confusion_matrix(true_labels, pred_labels) 

print ('Confusion Matrix :')

def plot_confusion_matrix(cm, target_names, title='Confusion Matrix', cmap=plt.cm.Greens):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    width, height = cm.shape

    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')


plot_confusion_matrix(results, ['Normal','Anomaly'])

In [ ]:
def reduce_anomalies(df, pct_anomalies=.01):
    labels = df['label'].copy()
    is_anomaly = labels != 'normal.'
    num_normal = np.sum(~is_anomaly)
    num_anomalies = int(pct_anomalies * num_normal)
    all_anomalies = labels[labels != 'normal.']
    anomalies_to_keep = np.random.choice(all_anomalies.index, size=num_anomalies, replace=False)
    anomalous_data = df.iloc[anomalies_to_keep].copy()
    normal_data = df[~is_anomaly].copy()
    new_df = pd.concat([normal_data, anomalous_data], axis=0)
    return new_df

In [ ]:
df = reduce_anomalies(df)

In [ ]:
# Rerun the cell from start. 